# 8. Plotting Suggesting Trades

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import timedelta

# Create sample data if not already available
np.random.seed(42)
n_points = 200
dates = pd.date_range('2023-01-01', periods=n_points, freq='1h')
close_prices = 16000 + np.cumsum(np.random.randn(n_points) * 100)

best_coin = 'BTC'
best_coin_df = pd.DataFrame({
    'timestamp': dates,
    'open': close_prices - np.abs(np.random.randn(n_points) * 50),
    'high': close_prices + np.abs(np.random.randn(n_points) * 50),
    'low': close_prices - np.abs(np.random.randn(n_points) * 50),
    'close': close_prices,
    'volume': np.random.rand(n_points) * 1000
})

# Add EMA columns
best_coin_df['EMA_50'] = best_coin_df['close'].ewm(span=50, adjust=False).mean()
best_coin_df['EMA_20'] = best_coin_df['close'].ewm(span=20, adjust=False).mean()

# Add signal column
best_coin_df['signal'] = np.where(best_coin_df['EMA_20'] > best_coin_df['EMA_50'], 'Long', 'Short')

# Add take_profit and stop_loss
best_coin_df['take_profit'] = best_coin_df['close'] * 1.02  # 2% above current price
best_coin_df['stop_loss'] = best_coin_df['close'] * 0.98    # 2% below current price

best_metric = 'EMA_20'

def plot_candlestick_with_signals(df, metric_list, title, plot_positions=True, future_time_minutes=300, height=500):
   
    future_time = df['timestamp'].iloc[-1] + timedelta(minutes=future_time_minutes)

    # Check if OHLC data exists, otherwise use line chart
    if 'open' in df.columns and 'high' in df.columns and 'low' in df.columns:
        fig = go.Figure(data=[go.Candlestick(x=df['timestamp'],
                                             open=df['open'],
                                             high=df['high'],
                                             low=df['low'],
                                             close=df['close'])])
    else:
        # If OHLC data missing, use line chart for close price
        fig = go.Figure(data=[go.Scatter(x=df['timestamp'],
                                         y=df['close'],
                                         mode='lines',
                                         name='Close Price')])

    for metric in metric_list:
        fig.add_trace(go.Scatter(x=df['timestamp'], y=df[metric], mode='lines', name=metric))

    if plot_positions and 'signal' in df.columns:
        last_signal_row = df.iloc[-1]
        if last_signal_row['signal'] == 'Long':
            fig.add_shape(type="rect",
                          x0=last_signal_row['timestamp'], x1=future_time,
                          y0=last_signal_row[metric_list[0]], y1=last_signal_row['take_profit'],
                          fillcolor="green", opacity=0.2, line_width=0)
            fig.add_shape(type="rect",
                          x0=last_signal_row['timestamp'], x1=future_time,
                          y0=last_signal_row[metric_list[0]], y1=last_signal_row['stop_loss'],
                          fillcolor="red", opacity=0.2, line_width=0)
        elif last_signal_row['signal'] == 'Short':
            fig.add_shape(type="rect",
                          x0=last_signal_row['timestamp'], x1=future_time,
                          y0=last_signal_row[metric_list[0]], y1=last_signal_row['stop_loss'],
                          fillcolor="red", opacity=0.2, line_width=0)
            fig.add_shape(type="rect",
                          x0=last_signal_row['timestamp'], x1=future_time,
                          y0=last_signal_row[metric_list[0]], y1=last_signal_row['take_profit'],
                          fillcolor="green", opacity=0.2, line_width=0)

    # Enable zoom and pan for the y-axis
    fig.update_layout(
        title=title,
        xaxis_title='Time',
        yaxis_title='Price',
        height=height,
        yaxis=dict(
            fixedrange=False,  # Allow vertical zoom
            rangemode='normal',  # Ensure y-axis is dynamic
        ),
        xaxis=dict(
            rangeslider=dict(visible=True),  # Horizontal range slider
        )
    )

    return fig

In [4]:
plot_candlestick_with_signals(best_coin_df, [best_metric], f"{best_coin} with Suggested Position", future_time_minutes=300, height=400)